<a href="https://colab.research.google.com/github/ALejandroCh17/Tennis-Ball-Robot/blob/daniel_branch/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
for label_path in train_label_paths + validation_label_paths + test_label_paths:
    with open(label_path, 'r') as file:
        label = np.genfromtxt(file, dtype='float32')
        if label.shape != (4,):  # Adjust this condition based on your expected label shape
            print(f"Found label with unexpected shape in file {label_path}: {label.shape}")


NameError: name 'train_label_paths' is not defined

In [5]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

# Define base directory
base_dir = '/content/drive/My Drive/capstone_dataset_3/'

# Define paths for train, validation, and test sets
train_dir = base_dir + 'train/'
validation_dir = base_dir + 'valid/'
test_dir = base_dir + 'test/'

# Define image and label directories
train_images_dir = train_dir + 'images/'
train_labels_dir = train_dir + 'labels/'
validation_images_dir = validation_dir + 'images/'
validation_labels_dir = validation_dir + 'labels/'
test_images_dir = test_dir + 'images/'
test_labels_dir = test_dir + 'labels/'

def parse_label_file(label_file_path):
    # Convert the input to string, because file paths are expected to be strings
    label_file_path = label_file_path.decode('utf-8')
    # Load all data from the label file
    data = np.genfromtxt(label_file_path, dtype='float32')
    # Select the first row if there are multiple, otherwise use the data as is
    if data.ndim > 1:
        #print(f"Multiple labels found in {label_file_path}, using the first one.")
        data = data[0, :]
    # Skip the first element (class id) and keep the last four elements (bounding box coordinates)
    bbox = data[1:]
    # Ensure that the label now contains exactly 4 elements
    if bbox.shape[0] != 4:
        raise ValueError(f'Label file {label_file_path} does not have 4 bounding box values: {bbox}')
    return bbox





def load_and_preprocess(image_file_path, label_file_path):
    img = tf.io.read_file(image_file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.image.convert_image_dtype(img, tf.float32)
    img.set_shape([224, 224, 3])

    label = tf.numpy_function(parse_label_file, [label_file_path], tf.float32)
    label.set_shape([4])
    return img, label



# Function to create a dataset from image and label paths
def create_dataset(image_paths, label_paths):
    image_paths = tf.constant(image_paths)
    label_paths = tf.constant(label_paths)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

# Collecting image and label file paths
train_image_paths = [os.path.join(train_images_dir, img) for img in os.listdir(train_images_dir)]
train_label_paths = [os.path.join(train_labels_dir, os.path.splitext(img)[0] + '.txt') for img in os.listdir(train_images_dir)]

validation_image_paths = [os.path.join(validation_images_dir, img) for img in os.listdir(validation_images_dir)]
validation_label_paths = [os.path.join(validation_labels_dir, os.path.splitext(img)[0] + '.txt') for img in os.listdir(validation_images_dir)]

test_image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir)]
test_label_paths = [os.path.join(test_labels_dir, os.path.splitext(img)[0] + '.txt') for img in os.listdir(test_images_dir)]

# Creating datasets
train_dataset = create_dataset(train_image_paths, train_label_paths)
validation_dataset = create_dataset(validation_image_paths, validation_label_paths)
test_dataset = create_dataset(test_image_paths, test_label_paths)

# Batching the datasets
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Assuming create_model function is defined and compiled
input_shape = (224, 224, 3)  # Update this if your images have a different shape
model = create_model(input_shape)

# Train the model
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset
)
"""
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

# Plotting training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
"""

NameError: name 'create_model' is not defined